In [1]:
import pandas as pd

In [2]:
df= pd.read_csv("train.csv.zip")

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [5]:
df= df.dropna()

In [6]:
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [7]:
X = df.drop('label',axis=1)

In [8]:
y = df['label']

In [9]:
df.label

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 18285, dtype: int64

In [10]:
X.shape

(18285, 4)

In [11]:
y.shape

(18285,)

In [12]:
import tensorflow as tf 

In [13]:
print(tf.__version__)

2.12.0


In [14]:
from tensorflow.keras.layers import Embedding 
from tensorflow.keras.layers import LSTM
from tensorflow.keras.utils import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import Dense

In [15]:
voc_size = 500

In [16]:
massages = X.copy()

In [17]:
massages["title"][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [18]:
massages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [19]:
 massages.reset_index(inplace=True)

In [20]:
massages

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
18280,20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
18281,20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
18282,20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
18283,20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [21]:
import nltk 
import re
from nltk.corpus import stopwords
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Data preprocessing

In [22]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [23]:
corpus = []
for i in range(0,len(massages)):
    review = re.sub('[^a-zA-Z]',' ',massages['title'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [24]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [25]:
onehot_repr = [one_hot(word ,voc_size) for word in corpus]
onehot_repr

[[149, 316, 377, 104, 383, 159, 479, 27, 230, 140],
 [8, 55, 461, 357, 270, 269, 335],
 [324, 408, 237, 99],
 [342, 323, 321, 118, 368, 390],
 [456, 270, 49, 364, 479, 261, 270, 449, 253, 197],
 [218, 475, 402, 497, 246, 301, 299, 220, 172, 492, 111, 30, 258, 174, 335],
 [188, 337, 324, 365, 387, 427, 96, 160, 103, 242, 142],
 [72, 142, 19, 205, 411, 7, 301, 420, 103, 242, 142],
 [292, 101, 48, 38, 345, 356, 447, 109, 301, 489],
 [77, 47, 222, 337, 58, 291, 82, 226],
 [433, 128, 53, 186, 410, 26, 484, 213, 468, 475, 26],
 [118, 284, 383, 356, 301, 411],
 [460, 201, 29, 348, 98, 128, 40, 27, 291],
 [435, 112, 205, 202, 481, 114, 275, 103, 242, 142],
 [10, 353, 423, 385, 132, 103, 242, 142],
 [69, 45, 89, 446, 380, 27, 56, 152, 462, 434],
 [391, 351, 55],
 [257, 239, 241, 37, 301, 201, 49, 335],
 [226, 394, 461, 74, 308, 456, 90, 472, 261],
 [145, 132, 301, 378, 351, 335],
 [148, 398, 124, 388, 169, 90, 344, 226, 378, 103, 242, 142],
 [316, 215, 297, 77, 437, 82, 65],
 [258, 100, 263, 35

In [26]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [27]:
onehot_repr[1]

[8, 55, 461, 357, 270, 269, 335]

# Embedding Representation

In [28]:
sent_length = 20
embedded_docs = pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[  0   0   0 ...  27 230 140]
 [  0   0   0 ... 270 269 335]
 [  0   0   0 ... 408 237  99]
 ...
 [  0   0   0 ... 103 242 142]
 [  0   0   0 ...  49 229 280]
 [  0   0   0 ... 487  72 438]]


In [29]:
embedded_docs[1]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         8,  55, 461, 357, 270, 269, 335])

# Creating Model

In [30]:
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Bidirectional 
embedding_vector_features = 40 
model = Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(100)))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [31]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            20000     
                                                                 
 dropout (Dropout)           (None, 20, 40)            0         
                                                                 
 bidirectional (Bidirectiona  (None, 200)              112800    
 l)                                                              
                                                                 
 dropout_1 (Dropout)         (None, 200)               0         
                                                                 
 dense (Dense)               (None, 1)                 201       
                                                                 
Total params: 133,001
Trainable params: 133,001
Non-trainable params: 0
__________________________________________________

In [32]:
len(embedded_docs),y.shape

(18285, (18285,))

In [33]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(y)

X_final.shape,y_final.shape

((18285, 20), (18285,))

In [34]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_final,y_final,test_size=0.33,random_state=42)

# Early Stopping

In [35]:
import tensorflow as tf
early_stopping= tf.keras.callbacks.EarlyStopping(monitor="val_loss",min_delta=0.001,patience=20,verbose=1,mode='auto',
                                                baseline=None,restore_best_weights=False)

# Model Training

In [36]:
model.fit(X_train,y_train,validation_data = (X_test,y_test),epochs=100,batch_size = 100,callbacks=early_stopping)

Epoch 1/100
123/123 [==============================] - 7s 37ms/step - loss: 0.3765 - accuracy: 0.8191 - val_loss: 0.2396 - val_accuracy: 0.9009
Epoch 2/100
123/123 [==============================] - 4s 36ms/step - loss: 0.2296 - accuracy: 0.9080 - val_loss: 0.2365 - val_accuracy: 0.9056
Epoch 3/100
123/123 [==============================] - 4s 36ms/step - loss: 0.2129 - accuracy: 0.9136 - val_loss: 0.2303 - val_accuracy: 0.9031
Epoch 4/100
123/123 [==============================] - 4s 34ms/step - loss: 0.2024 - accuracy: 0.9158 - val_loss: 0.2288 - val_accuracy: 0.9046
Epoch 5/100
123/123 [==============================] - 4s 33ms/step - loss: 0.1937 - accuracy: 0.9216 - val_loss: 0.2237 - val_accuracy: 0.9087
Epoch 6/100
123/123 [==============================] - 4s 34ms/step - loss: 0.1846 - accuracy: 0.9229 - val_loss: 0.2185 - val_accuracy: 0.9057
Epoch 7/100
123/123 [==============================] - 5s 37ms/step - loss: 0.1777 - accuracy: 0.9267 - val_loss: 0.2237 - val_accuracy:

# performance matrics & Accuracy

In [37]:
y_pred = model.predict(X_test)

189/189 [==============================] - 1s 4ms/step


In [38]:
y_pred = np.where(y_pred>0.5,1,0)

In [39]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[3061,  358],
       [ 251, 2365]], dtype=int64)

In [40]:
from sklearn.metrics import accuracy_score 
accuracy_score(y_test,y_pred)

0.8990886495443248

In [41]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.92      0.90      0.91      3419
           1       0.87      0.90      0.89      2616

    accuracy                           0.90      6035
   macro avg       0.90      0.90      0.90      6035
weighted avg       0.90      0.90      0.90      6035



In [42]:
import joblib 
# Save the model as a pickle in a file 
joblib.dump(model,'saved_model1.pkl') 

['saved_model1.pkl']